In [1]:
#Importing necessary libraries
from sagemaker.huggingface import HuggingFace
import sagemaker
import os
from dotenv import load_dotenv
load_dotenv()

sagemaker.config INFO - Not applying SDK defaults from location: /Library/Application Support/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /Users/abhijitdeshpande/Library/Application Support/sagemaker/config.yaml


True

In [2]:
import os

# Remove proxy vars if not needed
os.environ.pop("HTTP_PROXY", None)
os.environ.pop("HTTPS_PROXY", None)

# Use the certifi-provided CA bundle
os.environ["AWS_CA_BUNDLE"] = "/Volumes/LaCie/Projects_portfolio/NLP/SupportIQ/venv/lib/python3.11/site-packages/certifi/cacert.pem"

# Test AWS call
import boto3
print(boto3.client('sts').get_caller_identity())

{'UserId': 'AIDA2PNZRCZDH2V4XL47W', 'Account': '720332985926', 'Arn': 'arn:aws:iam::720332985926:user/dev-user', 'ResponseMetadata': {'RequestId': '3952d7eb-cb5a-40c6-8af5-18c0535d912f', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '3952d7eb-cb5a-40c6-8af5-18c0535d912f', 'content-type': 'text/xml', 'content-length': '405', 'date': 'Sat, 05 Jul 2025 23:42:48 GMT'}, 'RetryAttempts': 0}}


In [3]:
#settinng up AWS profile
os.environ['AWS_PROFILE'] = 'dev-user'

session = sagemaker.Session()

In [4]:
#Defining hyper parameters
hyper_parameters = {
    'model_id': 'roberta-large',
    'task_type':'classification',
    'rank': 64,
    'alpha': 128,
    'dropout': 0.05,
    'bias': 'none',
    'lr': 1e-5,
    'epochs': 10,
    'wd': 0.01,
    'logging_steps': 50,
    'batch_size': 32,
    'save_steps': 200,
    'eval_steps': 100,
    'target_module': 'query,key,value',
    'early_stopping': 3,
    'max_length': 32
}

In [5]:
huggingface_estimator = HuggingFace(
            entry_point='train.py',
            source_dir=os.getenv("SOURCE_DIR"),
            role=os.getenv("ROLE"),
            instance_type='ml.g4dn.xlarge',
            instance_count=1,
            transformers_version='4.49.0',
            py_version='py311',
            pytorch_version='2.5.1',
            hyperparameters = hyper_parameters
)

In [1]:
# # upload data to S3
# input_data = session.upload_data(path='data/data_full.json',bucket='gen-ai-repository', key_prefix='finetuning/flan-t5/data')

In [6]:
huggingface_estimator.fit({'training': 's3://gen-ai-repository/finetuning/flan-t5/data/'})

INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.
INFO:sagemaker:Creating training-job with name: huggingface-pytorch-training-2025-07-05-23-42-49-059


2025-07-05 23:42:51 Starting - Starting the training job...
2025-07-05 23:43:05 Starting - Preparing the instances for training...
2025-07-05 23:43:33 Downloading - Downloading input data...
2025-07-05 23:43:58 Downloading - Downloading the training image........................
2025-07-05 23:48:36 Training - Training image download completed. Training in progress...bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
CUDA compat package should be installed for NVIDIA driver smaller than 550.144.03
Current installed NVIDIA driver version is 550.163.01
Skipping CUDA compat setup as newer NVIDIA driver is installed
2025-07-05 23:48:47,797 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
2025-07-05 23:48:47,816 sagemaker-training-toolkit INFO     No Neurons detected (normal if no neurons installed)
2025-07-05 23:48:47,826 sagemaker_pytorch_container.training INFO     Block until all host 

In [7]:
model_artifact_s3_uri = huggingface_estimator.model_data
print("Model Artifact S3 URI:")
print(model_artifact_s3_uri)

Model Artifact S3 URI:
s3://sagemaker-us-east-1-720332985926/huggingface-pytorch-training-2025-07-05-23-42-49-059/output/model.tar.gz
